In [18]:
import pandas as pd

# Loading the spam data
# ham is the label for non-spam messages
spam = pd.read_excel('test.xlsx')
spam.head(10)

,Status,Decisão
0,parcialmente provido,conhecido. parcialmente provido. unanime
1,desprovido,conhecido. improvido. unânime
2,parcialmente provido,conhecido. parcialmente provido. unânime
3,desprovido,conhecer em parte. negar provimento. unânime
4,desprovido,conhecer do recurso e negar-lhe provimento. un...
5,provido,conhecido. provido. unânime.
6,desprovido,apelação da autora e recurso adesivo da ré con...
7,desprovido,conhecido. preliminar rejeitada. prejudicial r...
8,parcialmente provido,dar parcial provimento ao recurso de m.c.b.s.r...
9,desprovido,conhecer do recurso e negar-lhe provimento.


In [20]:
import spacy
nlp = spacy.blank("pt")

# Create the TextCategorizer with exclusive classes and "bow" architecture
textcat = nlp.create_pipe(
              "textcat",
              config={
                "exclusive_classes": True,
                "architecture": "bow"})

# Add the TextCategorizer to the empty model
nlp.add_pipe(textcat)

In [21]:
# Add labels to text classifier
textcat.add_label("provido")
textcat.add_label("desprovido")
textcat.add_label("parcialmente provido")
textcat.add_label("parcialmente improvido")

1

In [25]:
train_texts = spam['Decisão'].values
train_labels = [{'cats': {'provido': label == 'provido',
                          'desprovido': label == 'desprovido',
                          'parcialmente provido': label == 'parcialmente provido',
                          'parcialmente improvido': label == 'parcialmente improvido',
                         }
                } 
                for label in spam['Status']]

In [28]:
train_data = list(zip(train_texts, train_labels))
train_data[:3]

[('conhecido. parcialmente provido. unanime',
  {'cats': {'provido': False,
    'desprovido': False,
    'parcialmente provido': True,
    'parcialmente improvido': False}}),
 ('conhecido. improvido. unânime',
  {'cats': {'provido': False,
    'desprovido': True,
    'parcialmente provido': False,
    'parcialmente improvido': False}}),
 ('conhecido. parcialmente provido. unânime',
  {'cats': {'provido': False,
    'desprovido': False,
    'parcialmente provido': True,
    'parcialmente improvido': False}})]

In [29]:
from spacy.util import minibatch

spacy.util.fix_random_seed(1)
optimizer = nlp.begin_training()

# Create the batch generator with batch size = 8
batches = minibatch(train_data, size=8)
# Iterate through minibatches
for batch in batches:
    # Each batch is a list of (text, label) but we need to
    # send separate lists for texts and labels to update().
    # This is a quick way to split a list of tuples into lists
    texts, labels = zip(*batch)
    nlp.update(texts, labels, sgd=optimizer)

In [30]:
import random

random.seed(1)
spacy.util.fix_random_seed(1)
optimizer = nlp.begin_training()

losses = {}
for epoch in range(10):
    random.shuffle(train_data)
    # Create the batch generator with batch size = 8
    batches = minibatch(train_data, size=8)
    # Iterate through minibatches
    for batch in batches:
        # Each batch is a list of (text, label) but we need to
        # send separate lists for texts and labels to update().
        # This is a quick way to split a list of tuples into lists
        texts, labels = zip(*batch)
        nlp.update(texts, labels, sgd=optimizer, losses=losses)
    print(losses)

{'textcat': 0.7457376020029187}
{'textcat': 1.571977543644607}
{'textcat': 2.39568333607167}
{'textcat': 2.662572581321001}
{'textcat': 2.8466447442770004}
{'textcat': 3.5637408513575792}
{'textcat': 3.649692595936358}
{'textcat': 4.426499838940799}
{'textcat': 4.99767283257097}
{'textcat': 5.7421208480373025}


In [31]:
texts = ["conhecer do recurso e negar-lhe provimento. unânime.", 
         "dar parcial provimento ao recurso de m.c.b.s.r.; conhecer parcialmente do recurso de a.a.m.i.s.a. e negar-lhe provimento. Unânime"]
docs = [nlp.tokenizer(text) for text in texts]
    
# Use textcat to get the scores for each doc
textcat = nlp.get_pipe('textcat')
scores, _ = textcat.predict(docs)

print(scores)

[[0.1942342  0.39787167 0.26488912 0.14300497]
 [0.15327866 0.26527014 0.47849807 0.10295311]]


In [32]:
# From the scores, find the label with the highest score/probability
predicted_labels = scores.argmax(axis=1)
print([textcat.labels[label] for label in predicted_labels])

['desprovido', 'parcialmente provido']
